In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re

def scrape_books():
    try:
        url = "http://books.toscrape.com/"
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, "html.parser")

        all_books = []

        books = soup.find_all("article", class_="product_pod")

        rating_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}

        for book in books:
            title = book.h3.a["title"]

            price_str = book.find("p", class_="price_color").text
            price = float(re.search(r"[\d.]+", price_str).group())

            rating_p = book.find("p", class_=re.compile("star-rating"))
            rating_text = rating_p["class"][1] # e.g., "Three"
            rating_num = rating_map.get(rating_text, 0) # Convert to number

            all_books.append({"title": title, "price": price, "rating": rating_num})

        csv_file = "books_data.csv"
        with open(csv_file, "w", newline="", encoding="utf-8") as file:
            writer = csv.DictWriter(file, fieldnames=["title", "price", "rating"])
            writer.writeheader()
            writer.writerows(all_books)

        print(f"Successfully scraped {len(all_books)} books and saved data to '{csv_file}'.")

        if all_books:
            # Calculate average price
            total_price = sum(book["price"] for book in all_books)
            average_price = total_price / len(all_books)

            # Find the best-rated book(s)
            max_rating = max(book["rating"] for book in all_books)
            best_rated_books = [book["title"] for book in all_books if book["rating"] == max_rating]

            print("\n--- Analysis ---")
            print(f"Average Price of Books: £{average_price:.2f}")
            print(f"Highest Rating: {max_rating} stars")
            print("Best-Rated Book(s):")
            for title in best_rated_books:
                print(f"- {title}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")


if __name__ == "__main__":
    scrape_books()

Successfully scraped 20 books and saved data to 'books_data.csv'.

--- Analysis ---
Average Price of Books: £38.05
Highest Rating: 5 stars
Best-Rated Book(s):
- Sapiens: A Brief History of Humankind
- Set Me Free
- Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)
- Rip it Up and Start Again
